### Modified Dempster-Shafer Mass Combination
*name: Austin Powell*
*contact: powellaus10@gmail.com*

Examples below demonstrate how our modified Dempster-Shafer algorithm could be used and some explanation of results.

In [1]:
# %load mass_comb_fxn.py

# masses are so far a set of 3 for each of the different combination of 0,1. In a dataset, the different masses will be represented
#  by the different columns of the dataset
#  - The entries for the masses are [mass_val0,mass_val1,mass_val01,mass_val_theta]
# prior0: the prior belief for class 0. Default is uninformative
# prior1: the prior belief for class 1. Default is uninformative
# prior01: the prior belief for both classes. Default is 0

# C0, C1, C01, C_theta are the modified DS beliefs where there is a prior. If no prior is specified, a uniform prior is assumed of equal
#  assignment


def safe_divide(numerator, denominator):
    if denominator <= 0:
        return numerator
    else:
        return numerator / denominator


# default prior values are uniform uninformative
def massComb(masses, prior0=0.5, prior1=0.5, prior01=0):

    # the space in the hypothesis space that is not in the evidence space
    prior_theta = 1 - (prior0 + prior1 + prior01)

    # since sum of m(A) must equal 1 there may be frame of descernment is
    # what's left over
    for i in range(len(masses)):
        masses[i].append(1 - sum(masses[i]))

    ##
    # PERFORM MASS COMBINATION
    ##
    print("The different mass functions are (the last row is ):")
    for row in masses:
        print(row)
    intrsxn_array_dim = len(masses[1])
    # set the dimenensions of the mass comb. matrix.
    intrsxn_array = [
        [0 for j in range(intrsxn_array_dim)] for i in range(intrsxn_array_dim)]

    ############### BEGIN: Combining all bpa's ##############################
    for i in range(0,len(masses)-1):
        if i == 0:
            print("first entry")
            K = 1  
            m0 = masses[0][0]
            m1 = masses[0][1]
            m01 = masses[0][2]
            m_theta = masses[0][3]
            print("First mass assignment. m0:", m0, "m1: ", m1,
                  "m01: ", m01, "m_theta: ", m_theta, "K: ", K)

        new_mass = [[m0, m1, m01, m_theta]]
        for col in range(intrsxn_array_dim):
            for row in range(intrsxn_array_dim):
                intrsxn_array[row][col] = new_mass[0][col]*masses[i + 1][row]

        # CALCULATE K - the measure of conflict
        K = intrsxn_array[0][1] + intrsxn_array[1][0]
        # Calculate belief functions
        m0 = (intrsxn_array[0][0] + intrsxn_array[2][0] + intrsxn_array[3][0]
            + intrsxn_array[0][2] + intrsxn_array[0][3]) / (1 - K)
        m1 = (intrsxn_array[1][1] + intrsxn_array[1][2] + intrsxn_array[1][3]
            + intrsxn_array[2][1] + intrsxn_array[3][1]) / (1 - K)
        m01 = intrsxn_array[2][3] / (1 - K)
        # normalize to emphasise agreement
        m_theta = intrsxn_array[3][3] / (1 - K)
        print("Next mass assignment. m0:", m0, "m1: ", m1,
              "m01: ", m01, "m_theta: ", m_theta, "K: ", K)
    ############### END: Combining all bpa's

    print("m0:", m0, "m1: ", m1, "m01: ",
          m01, "m_theta: ", m_theta, "K: ", K)
    # INCLUDE PRIOR INFORMATION
    print("\n")
    print("prior0: ", prior0, "prior1: ", prior1,
          "prior01: ", prior01, "prior_theta: ", prior_theta)
    # basic certainty assignment (bca) and normalize
    certainty_denominator = (safe_divide(numerator=m0, denominator=prior0) + safe_divide(numerator=m1, denominator=prior1)
                             +
                             safe_divide(
                                 numerator=m01, denominator=prior01)
                             + safe_divide(numerator=m_theta, denominator=prior_theta))
    print("certainty_denom: ", certainty_denominator)
    C0 = safe_divide(numerator=m0, denominator=prior0) / \
        certainty_denominator
    C1 = safe_divide(numerator=m1, denominator=prior1) / \
        certainty_denominator
    C01 = safe_divide(
        numerator=m01, denominator=prior01) / certainty_denominator
    C_theta = safe_divide(
        numerator=m_theta, denominator=prior_theta) / certainty_denominator
    print("C0: ", C0, "C1: ", C1, "C01: ", C01, "C_theta: ", C_theta)
    print("C0 + C1 + C01 + C_theta: ", C0 + C1 + C01 + C_theta, "\n")

    #print("inrsxn_array:", intrsxn_array[0][1])
    # Belief values: (not used yet, will be important later for multi-class
    # s)
    blf0 = m0
    blf1 = m1
    blf01 = m0 + m1 + m01

    # Plausible values:  (not used yet, will be important)
    plsb0 = m0 + m01 + m_theta
    plsb1 = m1 + m01 + m_theta
    plsb_theta = 1

    # Range of Uncertainty
    print("Range of uncertainty for case 0: ", "(", blf0, ",", plsb0, ")")
    print("Range of uncertainty for case 1: ", "(", blf1, ",", plsb1, ")")

    print("belief0:", blf0, "belief1", blf1, "belief01:", blf01)
    print("plausible0:", plsb0, "plausible1:", plsb1)
        # calculate K
    # K = (1 - m1)*(m2) + m1*(1 - m2) # K is a measure of conflict

        # joint_mass1 = (1 - m1) * (1 - m2) / (1- K)
        # joint_mass2 = (m1)*(m2) / (1 - K)
        # print (m1, m2,K, joint_mass1, joint_mass2)


# Example below uses values from Intelligent Control Systems
#print("\n" ,"\n","EXAMPLE FROM INTELLIGENT CONTROL SYSTEMS")
# massComb(masses=[[.3,.2,0,.5],[.2,0,.2,0.6]])

print("\n", "\n", )
print("EXAMPLE1: ")
# massComb(masses=[[.3, .7, 0], [.4, 0.6, 0]])
#massComb(masses=[[.3, .7, 0], [.4, 0.6, 0],[0.4,0.6,0]])
# massComb(masses=[[.3, .7, 0], [.4, 0.6, 0],[0.4,0.6,0],[0.2,0.8,0]])
#massComb(masses=[[.3, .7, 0], [.4, 0.6, 0],[0.4,0.6,0],[0.2,0.8,0],[0.2,0.8,0]])
massComb(masses=[[.3,.2,0],[.2,0,.2]])
# ONLY RUN FIRST EXAMPLE
exit()
# ONLY RUN FIRST EXAMPLE

# checks to make sure that
print("\n", "\n", "EXAMPLE1 w/ 50/50 prior: ")
massComb(masses=[[.3, .7, 0], [.4, 0.6, 0]], prior0=0.5, prior1=0.5)

print("\n", "\n", "EXAMPLE2: ")
massComb(masses=[[.3, 0.2, 0], [.2, 0, 0.2]])

print("\n", "\n", "EXAMPLE2a (with specified priors): ")
massComb(masses=[[.3, 0.2, 0], [.2, 0, 0.2]],
         prior0=0.25, prior1=0.25, prior01=0.25)


# fails on the one below...at least according to EvCombR package in R
print("\n", "\n", "EXAMPLE FROM EVCOMBR4: ")
massComb(masses=[[.3, 0.2, 0], [.2, 0, 0.2], [.6, 0.4, 0]])



 

EXAMPLE1: 
The different mass functions are (the last row is ):
[0.3, 0.2, 0, 0.5]
[0.2, 0, 0.2, 0.6]
first entry
First mass assignment. m0: 0.3 m1:  0.2 m01:  0 m_theta:  0.5 K:  1
Next mass assignment. m0: 0.4166666666666667 m1:  0.16666666666666669 m01:  0.10416666666666667 m_theta:  0.3125 K:  0.04000000000000001
m0: 0.4166666666666667 m1:  0.16666666666666669 m01:  0.10416666666666667 m_theta:  0.3125 K:  0.04000000000000001


prior0:  0.5 prior1:  0.5 prior01:  0 prior_theta:  0.0
certainty_denom:  1.5833333333333335
C0:  0.5263157894736842 C1:  0.2105263157894737 C01:  0.06578947368421052 C_theta:  0.19736842105263155
C0 + C1 + C01 + C_theta:  0.9999999999999999 

Range of uncertainty for case 0:  ( 0.4166666666666667 , 0.8333333333333334 )
Range of uncertainty for case 1:  ( 0.16666666666666669 , 0.5833333333333334 )
belief0: 0.4166666666666667 belief1 0.16666666666666669 belief01: 0.6875
plausible0: 0.8333333333333334 plausible1: 0.5833333333333334

 
 EXAMPLE1 w/ 50/50 pr

#### The following examples demonstrate how to input basic values into the the Modified DS

##### Example 1
The most basic demonstration of mass combination for 2 classes

In [3]:
# mass_value0 = 0.3, mass_value1 = 0.7
# Prior is not specified
massComb(masses = [[.3,.7,0],[.4,0.6,0]])

The different mass functions are (the last row is ):
[0.3, 0.7, 0, 0.0]
[0.4, 0.6, 0, 0.0]
first entry
First mass assignment. m0: 0.3 m1:  0.7 m01:  0 m_theta:  0.0 K:  1
Next mass assignment. m0: 0.2222222222222222 m1:  0.7777777777777777 m01:  0.0 m_theta:  0.0 K:  0.45999999999999996
m0: 0.2222222222222222 m1:  0.7777777777777777 m01:  0.0 m_theta:  0.0 K:  0.45999999999999996


prior0:  0.5 prior1:  0.5 prior01:  0 prior_theta:  0.0
certainty_denom:  1.9999999999999998
C0:  0.22222222222222224 C1:  0.7777777777777778 C01:  0.0 C_theta:  0.0
C0 + C1 + C01 + C_theta:  1.0 

Range of uncertainty for case 0:  ( 0.2222222222222222 , 0.2222222222222222 )
Range of uncertainty for case 1:  ( 0.7777777777777777 , 0.7777777777777777 )
belief0: 0.2222222222222222 belief1 0.7777777777777777 belief01: 0.9999999999999999
plausible0: 0.2222222222222222 plausible1: 0.7777777777777777


##### Example 1a
This next example demonstrates that the massCombination does not change when we specify unassuming priors (the values should be the same as above):

In [5]:
massComb(masses = [[.3,.7,0],[.4,0.6,0]], prior0 = 0.5, prior1 = 0.5)

The different mass functions are (the last row is ):
[0.3, 0.7, 0, 0, 0.0]
[0.4, 0.6, 0, 0, 0.0]
[0.12, 0.27999999999999997, 0.0, 0.0, 0.0]
[0.18, 0.42, 0.0, 0.0, 0.0]
[0.0, 0.0, 0, 0, 0.0]
[0.0, 0.0, 0, 0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0]
m0: 0.2222222222222222 m1:  0.7777777777777777 m01:  0.0 m_theta:  0.0 K:  0.45999999999999996


prior0:  0.5 prior1:  0.5 prior01:  0 prior_theta:  0.0
certainty_denom:  1.9999999999999998
C0:  0.22222222222222224 C1:  0.7777777777777778 C01:  0.0 C_theta:  0.0
C0 + C1 + C01 + C_theta:  1.0 

Range of uncertainty for case 0:  ( 0.2222222222222222 , 0.2222222222222222 )
Range of uncertainty for case 1:  ( 0.7777777777777777 , 0.7777777777777777 )
belief0: 0.2222222222222222 belief1 0.7777777777777777 belief01: 0.9999999999999999
plausible0: 0.2222222222222222 plausible1: 0.7777777777777777


##### Example 2
This example demonstrates that although values are specified for the 0 and 1 classes, these do not add up to 1 so the 'left-over' mass is assigned to the alternate hypothesis space.

In [ ]:
massComb(masses = [[.3,0.2,0],[.2,0,0.2]])

##### Example 2a
We are using the same mass values for classes 0 and 1, however as with example 1a we are explicitly specifying the prior. Note that the priors do not add up to 1 so that the remaining prior probability is assigned to alternate hypothesis space.

In [ ]:
massComb(masses = [[.3,0.2,0],[.2,0,0.2]], prior0 = 0.25, prior1 = 0.25, prior01 = 0.25)

#### Example 3
Previous examples show combining only two sources of information or "features". The below example demonstrates 4 "features" where there is an obvious bias in the information towards class 1. *Note: The belief for the result being class 0/1 is correct for values of 1 since we have not given the algorithm any indication that the truth could be outside of our null prediction*

In [6]:
# fails on the one below...at least according to EvCombR package in R
massComb(masses=[[.3, .7, 0], [.4, 0.6, 0],[0.4,0.6,0],[0.2,0.8,0],[0.2,0.8,0]])

The different mass functions are (the last row is ):
[0.3, 0.7, 0, 0.0]
[0.4, 0.6, 0, 0.0]
[0.4, 0.6, 0, 0.0]
[0.2, 0.8, 0, 0.0]
[0.2, 0.8, 0, 0.0]
[0.12, 0.27999999999999997, 0.0, 0.0]
[0.18, 0.42, 0.0, 0.0]
[0.0, 0.0, 0, 0.0]
[0.0, 0.0, 0.0, 0.0]
m0: 0.2222222222222222 m1:  0.7777777777777777 m01:  0.0 m_theta:  0.0 K:  0.45999999999999996


prior0:  0.5 prior1:  0.5 prior01:  0 prior_theta:  0.0
certainty_denom:  1.9999999999999998
C0:  0.22222222222222224 C1:  0.7777777777777778 C01:  0.0 C_theta:  0.0
C0 + C1 + C01 + C_theta:  1.0 

Range of uncertainty for case 0:  ( 0.2222222222222222 , 0.2222222222222222 )
Range of uncertainty for case 1:  ( 0.7777777777777777 , 0.7777777777777777 )
belief0: 0.2222222222222222 belief1 0.7777777777777777 belief01: 0.9999999999999999
plausible0: 0.2222222222222222 plausible1: 0.7777777777777777
